In [360]:
import urllib.request, json
import numpy as np
import pandas as pd
import logging

from random import randint
from time import sleep

In [361]:
# create logger
logger = logging.getLogger('openbilanci-extractor')
logger.setLevel(logging.DEBUG)

# create console handler and set level to debug
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)

# create formatter
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# add formatter to ch
ch.setFormatter(formatter)

# add ch to logger
logger.addHandler(ch)

In [418]:
df_sose = pd.read_csv('./data/opencivitas-spesa-storica+dati-irpef.csv')

In [419]:
df_sose.shape

(26702, 98)

In [421]:
type('i') == str

True

In [336]:
columns = ['COMUNE_CAT_COD','Denominazione Comune', 'ANNO', 'Sigla Provincia_y']

In [351]:
d_types = ['entrate', 'spese']
base_url = 'https://www.openbilanci.it/armonizzati/bilanci/%s-comune-%s/%s/dettaglio.json?year=2016&type=preventivo'

In [411]:
i = 0

Automatic pdb calling has been turned OFF


In [410]:
df_filter = df_sose[columns]

logger.info('extracting %d from opencivitas dataset' % df_sose.shape[0])
for r in df_filter.values:
    cod, city, year, province = r
    if city is not 'Nan':
        city = city.lower().replace(' ', '-')
        province = province.lower()
        city_url = base_url  % (city, province, d_types[0])
        
        filename = './data/bilanci/%s_%s.json' % (cod, city)
        
        try:
            sleep(randint(1,20))
            i +=1
            logger.info('processing url %s', city_url)
            with urllib.request.urlopen(city_url) as url:
                data = json.loads(url.read().decode())
                
                with open(filename, 'w') as o:
                    json.dump(data, o)
                    
                    logger.info('saved data into %s' % filename)
                
        except Exception as ex:
            logger.error('error %s for url %s' % (ex, city_url))

2018-01-11 18:35:38,008 - openbilanci-extractor - INFO - extracting 26702 from opencivitas dataset
2018-01-11 18:35:38,012 - openbilanci-extractor - INFO - processing url https://www.openbilanci.it/armonizzati/bilanci/abano-terme-comune-pd/entrate/dettaglio.json?year=2016&type=preventivo
2018-01-11 18:35:38,641 - openbilanci-extractor - INFO - saved data into ./data/bilanci/A001_abano-terme.json


In [ ]:
logger.info("finished extraction for %d urls" % i)